# **Part-3: Creating Consumption Layer DDLs**
## **Objectives of part3:**

* Part 3 covers the creation of tables in the consumption layer.
* The process includes loading data from the curated zone to the consumption zone as a one-time data load.
* Tables under different dimensions and the fact table will be verified.

**Data Loading to Consumption Zone:**

* Data will move from the curated zone to the consumption zone.
* Verification of tables and data will be conducted across dimensions and the fact table.


**Dimension and Fact Table Naming:**

* Dimension tables for customers and items are master data.
* The order fact table is aggregated on the day level.
* Different naming conventions may apply in specific projects.

**Web UI and Data Loading:**

* Start by creating the item and customer dimension tables.
* Follow with the creation of the order fact table.
* Business logic and additional transformations can be applied as needed.
*For simplicity, the focus is on demonstrating the data flow.

**Work Process:**

* Switch context to the consumption zone schema.
* Create the item dimension table with an auto-increment key and additional metadata.
* Create the customer dimension table with an auto-increment key and additional metadata.
* Create the order fact table, aggregating data at the date level.
* Validate tables using the "SHOW" command.
* Load data from the curated zone into the consumption zone for each table.

**Table Creation:**

* Item Dimension Table: Contains an auto-increment key and metadata from the curated item table.
* Customer Dimension Table: Includes an auto-increment key and metadata from the curated customer table.
* Order Fact Table: Aggregates data at the date level, referencing customer and item dimensions.

In [ ]:
use schema chk.consumption_zone;

In [ ]:
create or replace table item_dim (
        item_dim_key number autoincrement,
        item_id varchar(16),
        item_desc varchar,
        start_date date,
        end_date date,
        price number(7,2),
        item_class varchar(50),
        item_category varchar(50),
        added_timestamp timestamp default current_timestamp() ,
        updated_timestamp timestamp default current_timestamp() ,
        active_flag varchar(1) default 'Y'
    ) comment ='this is item table with in consumption schema';

In [ ]:
create or replace table customer_dim (
        customer_dim_key number autoincrement,
        customer_id varchar(18),
        salutation varchar(10),
        first_name varchar(20),
        last_name varchar(30),
        birth_day number,
        birth_month number,
        birth_year number,
        birth_country varchar(20),
        email_address varchar(50),
        added_timestamp timestamp default current_timestamp() ,
        updated_timestamp timestamp default current_timestamp() ,
        active_flag varchar(1) default 'Y'
    ) comment ='this is customer table with in consumption schema';

In [ ]:
 create or replace table order_fact (
      order_fact_key number autoincrement,
      order_date date,
      customer_dim_key number,
      item_dim_key number,
      order_count number,
      order_quantity number,
      sale_price number(20,2),
      disount_amt number(20,2),
      coupon_amt number(20,2),
      net_paid number(20,2),
      net_paid_tax number(20,2),
      net_profit number(20,2)
    ) comment ='this is order table with in consumption schema';


**Data Loading and Aggregation:**

* Load data from curated item, customer, and order tables to the corresponding dimension and fact tables.
* Aggregation is done for the order fact table based on order date, customer key, and item dimension.

Item Dimension First Time Load

In [ ]:
 insert into chk.consumption_zone.item_dim (
        item_id,
        item_desc,
        start_date,
        end_date,
        price,
        item_class,
        item_category)
    select
        item_id,
        item_desc,
        start_date,
        end_date,
        price,
        item_class,
        item_category
    from chk.curated_zone.curated_item;

Customer Dimension 1st Time Load

In [ ]:
insert into chk.consumption_zone.customer_dim (
        customer_id ,
        salutation ,
        first_name ,
        last_name ,
        birth_day ,
        birth_month ,
        birth_year ,
        birth_country ,
        email_address )
    select
        customer_id ,
        salutation ,
        first_name ,
        last_name ,
        birth_day ,
        birth_month ,
        birth_year ,
        birth_country ,
        email_address
  from chk.curated_zone.curated_customer;

Order Fact First Time Load

In [ ]:
insert into chk.consumption_zone.order_fact (
      order_date,
      customer_dim_key ,
      item_dim_key ,
      order_count,
      order_quantity ,
      sale_price ,
      disount_amt ,
      coupon_amt ,
      net_paid ,
      net_paid_tax ,
      net_profit
    )
    select
      co.order_date,
      cd.customer_dim_key ,
      id.item_dim_key,
      count(1) as order_count,
      sum(co.order_quantity) ,
      sum(co.sale_price) ,
      sum(co.disount_amt) ,
      sum(co.coupon_amt) ,
      sum(co.net_paid) ,
      sum(co.net_paid_tax) ,
      sum(co.net_profit)
    from chk.curated_zone.curated_order co
    join chk.consumption_zone.customer_dim cd on cd.customer_id = co.customer_id
    join chk.consumption_zone.item_dim id on id.item_id = co.item_id and id.item_desc = co.item_desc and id.end_date is null
    group by
        co.order_date,
        cd.customer_dim_key ,
        id.item_dim_key
        order by co.order_date;

**Validation and Data Verification:**

* Verify tables for customer dimension, item dimension, and order fact.
* Validate data in terms of the number of rows and content.
* Copy history is visible in Snow Site, showing the data copying process.

**Conclusion of Part 3:**

* Part 3 successfully creates permanent tables in the consumption zone.
* Data is loaded from the curated zone with one-time data load.
* Item and customer dimension tables are established, along with an aggregated order fact table.
* Data aggregation is performed at the date level.
* Snow Site displays copy history for data copied through SQL commands.
* Part 4 will encompass creating tasks and streams for continuous data loading and changes capturing.





